In [2]:
import pandas as pd
import numpy as np



# Explo


In [5]:
from itertools import product
import pandas as pd
def create_dataset_for_explo_physical(path='data/Sanitized_Master_Physical_stores_26062024.csv'):
       df = pd.read_csv(path,index_col=0)
       df['Day of Date'] = pd.to_datetime(df['Day of Date'])

       filtered = df[['SKU', 'Day of Date','Cash Unit Sale',
              'Sale of Credit Units', 'Cash Sale Pesos', 'Sale Pesos Credit',
              'Cash Cost of Sale', 'Cost of Sale Credit', 'Elektra Cash Price\n(De)',
              'Elektra Credit Price\n(De)', 'Cash Price To Elektra (A)',
              'Price To Elektra Credit (A)','Pilot/Control','GROSS Contribution (Revenue-Cost of Sale)','Gross margin%']]
       filtered = filtered[filtered['Day of Date']>'2023-01-01']

       sku_list = filtered['SKU'].unique()
       date_range = pd.date_range(start=filtered['Day of Date'].min(), end=filtered['Day of Date'].max(), freq='D')
       control_treatment_categories = ['CONTROL', 'PILOTO']


       all_combinations = pd.DataFrame(list(product(sku_list,
                                                 control_treatment_categories,
                                                 date_range)),
                                   columns=['SKU', 'Pilot/Control', 'Day of Date'])

       merged_df = pd.merge(all_combinations, filtered, on=['SKU', 'Pilot/Control', 'Day of Date'], how='left')

       merged_df.sort_values(by=['SKU', 'Day of Date'], inplace=True)
       merged_df.fillna({
              'Cash Unit Sale': 0,
              'Sale of Credit Units': 0,
              'Cash Sale Pesos':0 ,
              'Sale Pesos Credit': 0
              }, inplace=True)


       merged_df = merged_df.sort_values(by=['SKU', 'Pilot/Control', 'Day of Date'])
       
       merged_df = merged_df.groupby(['SKU', 'Pilot/Control'], as_index=False).apply(lambda group: group.ffill().bfill())

       merged_df.reset_index(drop=True,inplace=True)
       
       # sku_map = pd.read_csv('data/sku_product_name_map.csv',index_col=0)
       sku_map = pd.read_csv('data/sku_product_name_map.csv',index_col=0)
       combined = pd.merge(merged_df,sku_map,on='SKU',how='outer')
       
       combined.to_csv("data/Raw_Data_Physical_Explo.csv")


/var/folders/kq/5tmn8y9n1fzgmrhxm2d96s700000gn/T/ipykernel_7093/3185533324.py:4: DtypeWarning: Columns (4,30,31,32,33,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path,index_col=0)


In [ ]:
def create_dataset_for_explo_digital(path='data/Sanitized_Master_Physical_stores_26062024.csv'):
       df = pd.read_csv(path,index_col=0)
       # df = pd.read_csv('s3://elektra-data/transformed/Sanitized_Master_Digital_stores_21072024.csv')
       df['Day of Date'] = pd.to_datetime(df['Day of Date'].str.split().str[0], format='%Y-%m-%d')
       filtered = df[['SKU', 'Day of Date','Cash Unit Sale',
                     'Sale of Credit Units', 'Cash Sale Pesos', 'Sale Pesos Credit',
                     'Cash Cost of Sale', 'Cost of Sale Credit', 'Elektra Cash Price\n(De)',
                     'Elektra Credit Price\n(De)', 'Cash Price To Elektra (A)',
                     'Price To Elektra Credit (A)','Pilot/Control','GROSS Contribution (Revenue-Cost of Sale)','Gross margin%']]
       filtered = filtered[filtered['Day of Date']>'2023-01-01']
       filtered

       sku_list = filtered['SKU'].unique()
       date_range = pd.date_range(start=filtered['Day of Date'].min(), end=filtered['Day of Date'].max(), freq='D')
       control_treatment_categories = ['DIGITAL']


       all_combinations = pd.DataFrame(list(product(sku_list,
                                                 
                                                 date_range)),
                                   columns=['SKU', 'Day of Date'])

       merged_df = pd.merge(all_combinations, filtered, on=['SKU', 'Day of Date'], how='left')
       merged_df
       merged_df.sort_values(by=['SKU', 'Day of Date'], inplace=True)
       merged_df.fillna({
              'Cash Unit Sale': 0,
              'Sale of Credit Units': 0,
              'Cash Sale Pesos':0 ,
              'Sale Pesos Credit': 0
              }, inplace=True)


       merged_df = merged_df.sort_values(by=['SKU', 'Day of Date'])

       merged_df = merged_df.groupby(['SKU'], as_index=False).apply(lambda group: group.ffill().bfill())

       merged_df.reset_index(drop=True,inplace=True)
       merged_df
       sku_map = pd.read_csv('data/sku_product_name_map.csv',index_col=0)
       combined = pd.merge(merged_df,sku_map,on='SKU',how='left')

       combined.to_csv("data/Raw_Data_Digital_Explo.csv")

In [ ]:
create_dataset_for_explo_digital('s3://elektra-data/transformed/Sanitized_Master_Digital_stores_21072024.csv')
create_dataset_for_explo_physical('s3://elektra-data/transformed/Sanitized_Master_Physical_stores_21072024.csv')


In [6]:
# sku_map = pd.read_csv('data/helper/sku_product_name_map.csv',index_col=0)
# merged_df = pd.read_csv("data/Raw_Data_Physical_Explo.csv",index_col=0)
# combined = pd.merge(merged_df,sku_map,on='SKU',how='outer')
# combined
# combined.to_csv("data/Raw_Data_Physical_Explo.csv")

# Join Predictions with actual

In [29]:

preds_df = pd.read_csv('Experiment_Jun07_EKT.csv')
# preds_df = preds_df.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
preds_df

,Date,SKU,Best Price Credit A,Best Price Cash A,Best Price Credit DE,Best Price Cash DE,Credit Sales Prediction,Predicted Revenue
0,2024-06-07,31059203,9699.03,10298.97,16488.351,13388.661,0,0.0
1,2024-06-07,31058953,3394.03,3603.97,5769.851,4685.161,0,0.0
2,2024-06-07,31058977,5625.03,5375.57,9562.551,6988.241,0,0.0
3,2024-06-07,31058721,3394.03,3603.97,5769.851,4685.161,0,0.0
4,2024-06-07,31058984,6983.03,7414.97,11871.151,9639.461,0,0.0
...,...,...,...,...,...,...,...,...
1531,2024-06-14,31057028,3394.00,3499.00,5770.000,4549.000,0,0.0
1532,2024-06-14,31051402,2424.00,2499.00,4121.000,3249.000,0,0.0
1533,2024-06-14,31053437,1552.00,1600.00,2638.000,2080.000,0,0.0
1534,2024-06-14,31045187,2212.00,2280.00,3760.000,2964.000,0,0.0


In [30]:
actuals_df = pd.read_csv('data/Raw_Data_Physical_Explo.csv',index_col=0)

actuals_df['Day of Date'] = pd.to_datetime(actuals_df['Day of Date'])
date_ranges = [
    (pd.Timestamp('2024-06-04'), pd.Timestamp('2024-06-06'), pd.Timestamp('2024-06-04')),
    (pd.Timestamp('2024-06-07'), pd.Timestamp('2024-06-10'), pd.Timestamp('2024-06-07')),
    
]

actuals_df['Pricing_date'] = actuals_df['Day of Date']


for start_date, end_date, label_date in date_ranges:
    actuals_df['Pricing_date'] = actuals_df['Pricing_date'].apply(lambda x: label_date if start_date <= x <= end_date else x)
actuals_df = actuals_df[actuals_df['Day of Date']>=pd.Timestamp('2024-06-04')]
actuals_df.columns

Index(['SKU', 'Pilot/Control', 'Day of Date', 'Cash Unit Sale',
       'Sale of Credit Units', 'Cash Sale Pesos', 'Sale Pesos Credit',
       'Cash Cost of Sale', 'Cost of Sale Credit', 'Elektra Cash Price\n(De)',
       'Elektra Credit Price\n(De)', 'Cash Price To Elektra (A)',
       'Price To Elektra Credit (A)', 'Product Name', 'Brand', 'Model',
       'Carrier', 'Introductory_price', 'price_segment', 'Pricing_date'],
      dtype='object')

In [31]:
actuals_df = actuals_df[actuals_df['Pilot/Control']=='PILOTO']
actuals_df.columns

Index(['SKU', 'Pilot/Control', 'Day of Date', 'Cash Unit Sale',
       'Sale of Credit Units', 'Cash Sale Pesos', 'Sale Pesos Credit',
       'Cash Cost of Sale', 'Cost of Sale Credit', 'Elektra Cash Price\n(De)',
       'Elektra Credit Price\n(De)', 'Cash Price To Elektra (A)',
       'Price To Elektra Credit (A)', 'Product Name', 'Brand', 'Model',
       'Carrier', 'Introductory_price', 'price_segment', 'Pricing_date'],
      dtype='object')

In [9]:
actuals_df[(actuals_df['Pricing_date']=='2024-06-07') & (actuals_df['SKU']==31057241)]['Sale of Credit Units'].sum()

28.0

In [10]:

def mode(x):
        return x.mode().max() if not x.mode().empty else np.nan

grouped = actuals_df.groupby(['Pricing_date','SKU']).agg({
    'Cash Unit Sale':'sum',
    'Sale of Credit Units':'sum',
    'Cash Cost of Sale' :mode,
    'Cost of Sale Credit': mode,
    'Elektra Cash Price\n(De)': mode,
       'Elektra Credit Price\n(De)': mode,
       'Cash Price To Elektra (A)': mode,
       'Price To Elektra Credit (A)': mode,
    
}).reset_index()
grouped

,Pricing_date,SKU,Cash Unit Sale,Sale of Credit Units,Cash Cost of Sale,Cost of Sale Credit,Elektra Cash Price\n(De),Elektra Credit Price\n(De),Cash Price To Elektra (A),Price To Elektra Credit (A)
0,2024-06-04,31043249,0.0,0.0,-1875.00,0.00,2899.0,0.0,870.0,0.0
1,2024-06-04,31044050,0.0,0.0,0.00,-5818.32,0.0,8999.0,0.0,2700.0
2,2024-06-04,31044759,0.0,0.0,-4848.49,0.00,7499.0,0.0,2250.0,0.0
3,2024-06-04,31044766,0.0,0.0,0.00,-5301.11,0.0,8499.0,0.0,2550.0
4,2024-06-04,31044773,0.0,0.0,-560.34,0.00,6499.0,0.0,1625.0,0.0
...,...,...,...,...,...,...,...,...,...,...
987,2024-06-07,31059240,0.0,0.0,0.00,-6723.46,0.0,5965.0,0.0,5965.0
988,2024-06-07,31059241,0.0,0.0,0.00,-6982.06,0.0,9999.0,0.0,7649.0
989,2024-06-07,31059260,0.0,0.0,0.00,-4887.23,0.0,6999.0,0.0,5999.0
990,2024-06-07,31059261,0.0,0.0,0.00,-6982.06,0.0,9999.0,0.0,7999.0


In [12]:
preds_df

,Date,SKU,Best Price Credit A,Best Price Cash A,Best Price Credit DE,Best Price Cash DE,Credit Sales Prediction,Predicted Revenue
0,2024-06-07,31059203,9699.030000,10298.970000,16488.351000,13388.661000,0.0,0.000000
1,2024-06-07,31058953,3394.030000,3603.970000,5769.851000,4685.161000,0.0,0.000000
2,2024-06-07,31058977,5625.030000,5375.570000,9562.551000,6988.241000,0.0,0.000000
3,2024-06-07,31058721,3394.030000,3603.970000,5769.851000,4685.161000,0.0,0.000000
4,2024-06-07,31058984,6983.030000,7414.970000,11871.151000,9639.461000,0.0,0.000000
...,...,...,...,...,...,...,...,...
507,2024-06-07,31057384,1475.839711,1659.000085,2508.927509,2156.700110,4.0,5903.358844
508,2024-06-07,31055343,2014.701689,3443.318648,3424.992871,4476.314242,10.0,20147.016890
509,2024-06-07,31057661,2835.781046,2856.219289,4820.827778,3713.085076,5.0,14178.905230
510,2024-06-07,31057150,4108.562887,5719.382331,6984.556908,7435.197030,6.0,24651.377322


In [13]:
preds_df['Date'] = pd.to_datetime(preds_df['Date'])
grouped['Pricing_date'] = pd.to_datetime(grouped['Pricing_date'])
combined = pd.merge(preds_df,grouped, how='left',left_on=['Date','SKU'], right_on=['Pricing_date','SKU'])

In [16]:
combined[(combined['Pricing_date']=='2024-06-07') & (combined['SKU']==31057241)]

,Date,SKU,Best Price Credit A,Best Price Cash A,Best Price Credit DE,Best Price Cash DE,Credit Sales Prediction,Predicted Revenue,Pricing_date,Cash Unit Sale,Sale of Credit Units,Cash Cost of Sale,Cost of Sale Credit,Elektra Cash Price\n(De),Elektra Credit Price\n(De),Cash Price To Elektra (A),Price To Elektra Credit (A)
444,2024-06-07,31057241,2442.121317,4229.218431,4151.606239,5497.98396,18.0,43958.183706,2024-06-07,34.0,28.0,-3490.68,0.0,4999.0,0.0,1999.0,0.0


In [44]:
combined.fillna({
            'Cash Unit Sale': 0,
            'Sale of Credit Units':0
        }, inplace=True)
combined = combined.drop(['Pricing_date'],axis=1)
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Date                         512 non-null    datetime64[ns]
 1   SKU                          512 non-null    int64         
 2   Best Price Credit A          510 non-null    float64       
 3   Best Price Cash A            510 non-null    float64       
 4   Best Price Credit DE         510 non-null    float64       
 5   Best Price Cash DE           510 non-null    float64       
 6   Credit Sales Prediction      512 non-null    float64       
 7   Predicted Revenue            512 non-null    float64       
 8   Cash Unit Sale               512 non-null    float64       
 9   Sale of Credit Units         512 non-null    float64       
 10  Cash Cost of Sale            490 non-null    float64       
 11  Cost of Sale Credit          490 non-null    

In [45]:
combined.to_csv('Predictions_View.csv')